In [ ]:
# default_exp exec.plotting

In [ ]:
# export
from montlake.atomgeom.features import get_features,get_D_feats_feats
from montlake.atomgeom.utils import get_atoms_4
from montlake.simulations.rigidethanol import get_rigid_ethanol_data
from montlake.utils.utils import get_234_indices, get_atoms3_full, get_atoms4_full, data_stream_custom_range, get_cosines
from montlake.geometry.geometry import get_geom, get_wlpca_tangent_sel, get_rm_tangent_sel
from montlake.gradients.estimate import get_grads_pullback
from montlake.statistics.normalization import normalize_L212
from montlake.optimization.gradientgrouplasso import get_sr_lambda_parallel
from montlake.optimization.utils import get_selected_function_ids,get_selected_functions_lm2
from montlake.utils.replicates import Replicate, get_supports_brute
from montlake.plotting.manifolds import plot_manifold_2d,plot_manifold_featurespace
from montlake.plotting.flasso import plot_reg_path_ax_lambdasearch_customcolors_norm, plot_watch_custom
from megaman.embedding import SpectralEmbedding
from montlake.statistics.supportrecovery import get_min_min, get_mu_full_ind, get_kappa_s,get_gamma_max
from montlake.plotting.main import plot_experiment

import pandas as pd
import dill as pickle
import os
import sys
import numpy as np
import itertools
from itertools import permutations,combinations
from sklearn.decomposition import TruncatedSVD
import pathos
from pathos.multiprocessing import ProcessingPool as Pool

In [ ]:
# export
import numpy as np
import random
import argparse
import json
import sys
import scipy
from ..vendor.tfcompat.hparam import HParams
import os

def parse_args(args):
    parser= argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config (can override with cmd line args)")
    #parser.add_argument("--raw_data", help="Path to raw data")
    #parser.add_argument("--outdir", help="Path to save outputs")
    args = parser.parse_args(args)
    return args


In [ ]:
# export

try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
    
if __name__ == "__main__" and not IN_NOTEBOOK:
    
    np.random.seed(1234)
    random.seed(1234)
    args = parse_args(sys.argv[1:])
    config = {}
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))

    config.update(vars(args))
    hparams = HParams(**config)
    if not os.path.exists(hparams.outdir):
        os.makedirs(hparams.outdir)
        
    positions = np.load(hparams.position_file)
    for key in hparams.ground_truth.keys():
        if hparams.ground_truth[key] is None:
            hparams.ground_truth[key] = np.asarray([])
        hparams.ground_truth[key] = np.asarray(hparams.ground_truth[key], dtype = int)
        
    print('plotting',hparams.ground_truth)
    plot_experiment(result_file = hparams.result_file, 
                    positions = positions, 
                    d = hparams.d,
                    name = hparams.name, 
                    ncord = hparams.ncord,
                    embedding = hparams.embedding,
                    ground_truth = hparams.ground_truth, 
                    ground_truth_colors = hparams.ground_truth_colors,
                    outdir = hparams.outdir,
                    color_counts_all = hparams.color_counts_all,
                   colors_id_all = hparams.colors_id_all,
                   n_components = hparams.n_components,
                   ptsize = hparams.ptsize,
                   alpha = hparams.alpha)
        

In [11]:
import json
with open("/Users/samsonkoelle/tunatostada/experiments/configs_plotting/rigidethanol_diagram.json") as f:
    asdf = json.load(f)

In [14]:
    print('loading data')
    import dill as pickle
    with open("/Users/samsonkoelle/thesis_data/processed_data_2/rigidethanol/re_nonoise_diagram_mfresults_mflasso",'rb') as inp:
        results = pickle.load(inp, pickle.HIGHEST_PROTOCOL)


loading data


In [15]:
results['supports_lasso_values']

[(array([], dtype=float64),
  array([], dtype=float64),
  array([2.739275 , 1.5131816])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([3.11730394, 2.14200626])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([0.22783844, 3.07469065])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([3.10224651, 3.13579269])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([0.77866603, 1.87956494])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([0.08878809, 1.1354117 ])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([1.46961204, 2.01680351])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([1.59623417, 1.43885288])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([0.08712072, 2.13184689])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([2.97679625, 2.64488169])),
 (array([], dtype=float64),
  array([], dtype=float64),
  array([1.10748713, 3.137

In [12]:
config = {}
config.update(asdf)
config['outdir']
asdf

from montlake.vendor.tfcompat.hparam import HParams
hparams = HParams(**config)
config
hparams.outdir

'/Users/samsonkoelle/thesis_data/plots/rigidethanol'